In [ ]:
import numpy as np
import pandas as pd

def load_data(file_path):
    """Load dataset from a CSV file and prepare features and labels."""
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values  # Extract feature matrix
    y = data.iloc[:, -1].values  # Extract labels
    y = 2 * y - 1  # Transform labels to {-1, 1}
    # Add a bias term to the feature matrix
    X = np.c_[np.ones(X.shape[0]), X]
    return X, y

def shuffle_data(X, y):
    """Shuffle the dataset to randomize order."""
    indices = np.random.permutation(len(y))
    return X[indices], y[indices]

def hinge_loss(w, X, y, C):
    """Compute the hinge loss with regularization."""
    loss = np.maximum(0, 1 - y * np.dot(X, w))  # Hinge loss: max(0, 1 - y * (w · x))
    regularizer = 0.5 * np.sum(w[1:] ** 2) / C  # Regularization excludes the bias term
    return np.mean(loss) + regularizer

def subgradient(w, x, y, C):
    """Compute the subgradient for a single data point."""
    if 1 - y * np.dot(w, x) > 0:
        gradient = -y * x + np.concatenate(([0], w[1:])) / C  # Include regularization for weights
    else:
        gradient = np.concatenate(([0], w[1:])) / C
    return gradient

def svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0):
    """Train SVM using stochastic gradient descent."""
    m, n = X_train.shape  # Number of samples and features
    w = np.zeros(n)  # Initialize weights
    gamma = gamma_0  # Initial learning rate

    for epoch in range(1, max_epochs + 1):
        # Shuffle training data at the start of each epoch
        X_train, y_train = shuffle_data(X_train, y_train)

        for i in range(m):
            xi, yi = X_train[i], y_train[i]
            gradient = subgradient(w, xi, yi, C)
            w -= gamma * gradient  # Update weights using the gradient

        # Adjust learning rate with each epoch
        gamma = gamma_0 / (1 + epoch)

    # Compute final training and testing errors
    train_error = hinge_loss(w, X_train, y_train, C)
    test_error = hinge_loss(w, X_test, y_test, C)

    return w, train_error, test_error

train_file_path = "C:\\Users\\santhosh\\Downloads\\Dataset-svm\\bank-note\\train.csv"
test_file_path = "C:\\Users\\santhosh\\Downloads\\Dataset-svm\\bank-note\\test.csv"

# Load training and test datasets
X_train, y_train = load_data(train_file_path)
X_test, y_test = load_data(test_file_path)

# Define hyperparameters
C_values = [100 / 873, 500 / 873, 700 / 873]  # Regularization constants
max_epochs = 100  # Number of epochs
gamma_0 = 0.1  # Initial learning rate

# Train and evaluate the SVM for different C values
for C in C_values:
    print(f"\nTraining SVM with C={C}...\n")
    w, train_error, test_error = svm_sgd(X_train, y_train, X_test, y_test, C, max_epochs, gamma_0)

    # Display final results
    print(f"Final Training Error: {train_error}")
    print(f"Final Test Error: {test_error}")
    print(f"Weight Vector: {w}")
    print(f"Bias Term: {w[0]}")  # The bias term is the first element of the weight vector



Training SVM with C=0.1145475372279496...

Final Training Error: 0.6274299142920123
Final Test Error: 0.6200078351889313
Weight Vector: [ 0.13427096 -0.15304207 -0.12302393 -0.06497777 -0.01535376]
Bias Term: 0.13427095910214773

Training SVM with C=0.572737686139748...

Final Training Error: 0.3060021698065688
Final Test Error: 0.3121118736028021
Weight Vector: [ 0.61627551 -0.32607317 -0.17982843 -0.20292087 -0.01069369]
Bias Term: 0.6162755112297509

Training SVM with C=0.8018327605956472...

Final Training Error: 0.2597421193793294
Final Test Error: 0.26719359641019996
Weight Vector: [ 0.65818174 -0.35188487 -0.18731698 -0.20593004 -0.00785838]
Bias Term: 0.658181741814027
